In [ ]:
!pip install tensorflow==2.5 tensorflow-gpu==2.5
!pip install opencv-python mediapipe sklearn matplotlib

In [26]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [27]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [28]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [29]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [30]:
def draw_styled_landmarks(image, results):
#     FACEMESH_CONTOURS, FACEMESH_TESSELATION
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic. FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [31]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [32]:
PSLDataset = os.path.join('Dataset\PSL Dataset') 
GeneralDataset = os.path.join('Dataset\General Dataset') 
PersonalDataset = os.path.join('Dataset\Personal Dataset') 
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect 'Eye','Nose',
# actions = np.array(['Eye'])
actions = np.array(['Ankle','Blood','Fist','Heart','Jaw','Knuckle',
                    'Lips','Palm','Skull','Thumb'])

In [33]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [34]:
label_map = {label:num for num, label in enumerate(actions)}

In [35]:
label_map

{'Ankle': 0,
 'Blood': 1,
 'Fist': 2,
 'Heart': 3,
 'Jaw': 4,
 'Knuckle': 5,
 'Lips': 6,
 'Palm': 7,
 'Skull': 8,
 'Thumb': 9}

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import os

In [15]:
# !nvidia-smi
import subprocess

result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
output = result.stdout.decode('utf-8')

print(output)

Tue May 30 20:32:57 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.79                 Driver Version: 531.79       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 T...  WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   68C    P0               35W /  N/A|   1831MiB /  4096MiB |     80%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [13]:
# log_dir = os.path.join('Logs')
# tb_callback = TensorBoard(log_dir=log_dir)

NameError: name 'TensorBoard' is not defined

In [37]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [38]:
model.load_weights('Traingin_30Frames_tanh_testsplit0.05.h5')
# model.load_weights('mouth.h5')

In [39]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [40]:
def test_Video(video):
    def printFind(pred):
        for n in pred:
            print(actions[n])
        print('---------')

    # 1. New detection variables
    sequence = []
    sentence = []
    predictions = []
    threshold = 0.5
    print(actions)
    # testing on video
#     video = os.path.join(GeneralDataset,'Fist','9.mp4')
    cap = cv2.VideoCapture(video)

    #   Calculating frames
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = frame_count/fps
    duration = (duration/35)*1000
    count = 0
    print("Frame Count",frame_count)
    print("fps",fps)
    print("Duration",duration)
    print("Update: ",duration/35)
    print("Frame Time: ",duration)

    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()
    #         testin--------------
    #         hflip = iaa.Fliplr(p=1.0)
    #         frame = hflip.augment_image(frame)
    #         ------------------
    #       skipping frames
            cap.set(cv2.CAP_PROP_POS_MSEC, (count*duration))    
            # move the time
            success,image = cap.read()
            count += 1
            if success==False:
                break
            # Make detections
            image, results = mediapipe_detection(frame, holistic)
    #         print(results)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]

            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                printFind(predictions)
    #             print(predictions)
                print("Max: ",actions[np.argmax(res)])
                predictions.append(np.argmax(res))
    # #             predictions.append(np.argmax(res))
    #             predictions.append(res.argsort()[-3:][::-1])
    #             print(actions[res[0]],actions[res[1]],actions[res[2]])


            #3. Viz logic
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5: 
                    sentence = sentence[-5:]

                # Viz probabilities
    #             image = prob_viz(res, actions, image, colors)

            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

In [41]:
# testing on video
video = os.path.join(GeneralDataset,'Ankle','10.mp4')

test_Video(video)

['Ankle' 'Blood' 'Fist' 'Heart' 'Jaw' 'Knuckle' 'Lips' 'Palm' 'Skull'
 'Thumb']
Frame Count 134.0
fps 30.103490107783475
Duration 127.18031746031747
Update:  3.6337233560090705
Frame Time:  127.18031746031747
---------
Max:  Blood
Blood
---------
Max:  Blood
Blood
Blood
---------
Max:  Blood
Blood
Blood
Blood
---------
Max:  Thumb
Blood
Blood
Blood
Thumb
---------
Max:  Thumb
Blood
Blood
Blood
Thumb
Thumb
---------
Max:  Ankle


In [42]:
from scipy import stats

In [43]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [44]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

IndexError: list index out of range

<Figure size 1800x1800 with 0 Axes>

In [45]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
#             image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

Heart
Heart
Heart
Heart
Heart
Heart
Heart
Heart
Heart
Heart
Heart
Heart
Heart
Heart
Fist
Fist
Fist
Fist
Fist
Fist
Fist
Lips
Lips
Lips
Lips
Lips
Lips
Lips
Lips
Lips
Lips
Jaw
Blood
Blood
Blood
Blood
Blood
Blood
Blood
Blood
Blood
Fist
Fist
Fist
Fist
Lips
Lips
Lips
Lips
Lips
Lips
Lips
Lips
Lips
Lips
Lips
Lips
Lips
